In [32]:
import pandas as pd
import numpy as np
car_measurements = pd.read_csv("data/car_measurements_with_sites.csv")
sites = pd.read_csv("data/dcc_traffic_signals_20221130.csv")

In [13]:
car_measurements.head()
sites.head()

,SiteID,Site_Description_Cap,Site_Description_Lower,Region,Lat,Long,Site_Type
0,1,ABBEY ST @ MARLBOROUGH ST (LUAS),abbey st @ marlborough st (luas),CCITY,53.348754,-6.257607,SCATS Site
1,2,AMIENS ST @ SEVILLE PL,amiens st @ seville pl,CCITY,53.354711,-6.246679,SCATS Site
2,3,AMIENS ST @ TALBOT ST,amiens st @ talbot st,CCITY,53.351374,-6.250073,SCATS Site
3,4,ANNESLEY PL @ POPLAR ROW,annesley pl @ poplar row,NCITY,53.361145,-6.240513,SCATS Site
4,5,NCR @ AUGHRIM ST,ncr @ aughrim st,WCITY1,53.355797,-6.292406,SCATS Site


In [29]:
import folium
sitenumber = 420

#take a subset of the cardata with only Matched sites = site number
car_measurements_1 = car_measurements[car_measurements['Matched_Sites'] == f"[{sitenumber}]"]

#Take the site row from the sites data that matches the site in the car_measurements_1 data
sites_1 = sites[sites['SiteID']==sitenumber]

car_measurements_1.head()




,latitude,longitude,NO_ugm3,NO2_ugm3,O3_ugm3,CO_mgm3,CO2_mgm3,PM25_ugm3,date,Hour,Matched_Sites
16609,53.347048,-6.291006,48.711,NaN,NaN,NaN,NaN,NaN,2022-04-12,12,[420]
16610,53.347148,-6.290911,154.022,NaN,NaN,NaN,NaN,NaN,2022-04-12,12,[420]
16611,53.347236,-6.290836,345.036,NaN,NaN,NaN,NaN,NaN,2022-04-12,12,[420]
16612,53.347312,-6.290776,494.167,NaN,NaN,NaN,NaN,NaN,2022-04-12,12,[420]
16613,53.347377,-6.290717,516.123,NaN,NaN,NaN,NaN,NaN,2022-04-12,12,[420]


In [30]:
# Create a map centered at Dublin
m = folium.Map(location=[53.349805, -6.26031], zoom_start=12)

# Add points to the map
for idx, row in car_measurements_1.iterrows():
    folium.Circle
    
    folium.Circle(
        location=[row['latitude'], row['longitude']],
        radius=0.5,
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(m)

folium.Circle
    
folium.Circle(
    location=[sites_1['Lat'], sites_1['Long']],
    radius=1,
    color='red',
    fill=True,
    fill_color='red'
).add_to(m)
  
m

c:\Users\killi\anaconda3\envs\dsa2\Lib\site-packages\folium\utilities.py:94: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
c:\Users\killi\anaconda3\envs\dsa2\Lib\site-packages\folium\utilities.py:100: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
c:\Users\killi\anaconda3\envs\dsa2\Lib\site-packages\folium\utilities.py:102: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(x) for x in coords]


In [33]:
def parse_matched_sites(matched_sites):
    if matched_sites == '[]':
        return np.nan
    elif matched_sites.startswith('[') and matched_sites.endswith(']'):
        return [int(x.strip()) for x in matched_sites[1:-1].split(',')]
    else:
        raise ValueError("Invalid format for Matched_Sites column")

# Apply the function and explode the DataFrame
car_measurements['Matched_Sites'] = car_measurements['Matched_Sites'].apply(parse_matched_sites)
car_measurements = car_measurements.explode('Matched_Sites').reset_index(drop=True)

In [37]:
#Drop latitude and longitude columns
car_measurements2 = car_measurements.drop(columns=['latitude', 'longitude'])

# Average the measurements for a given site, Hour and date
averaged_car_measurements = car_measurements2.groupby(['Matched_Sites', 'Hour', 'date']).mean().reset_index()

In [40]:
averaged_car_measurements.shape

(43760, 9)

In [51]:
#sites kepp only 'Lat' and 'Long'
sites2 = sites[['SiteID', 'Lat', 'Long']]

# Merge the averaged measurements with the sites data
averaged_car_measurements2 = pd.merge(averaged_car_measurements, sites2, left_on='Matched_Sites', right_on='SiteID', how='inner')

#Drop Matched_Sites column
averaged_car_measurements2 = averaged_car_measurements2.drop(columns=['Matched_Sites'])

In [52]:
#Extract month and day of the week from the date column
averaged_car_measurements2['month'] = pd.to_datetime(averaged_car_measurements2['date']).dt.month
averaged_car_measurements2['day_of_week'] = pd.to_datetime(averaged_car_measurements2['date']).dt.dayofweek

,Hour,date,NO_ugm3,NO2_ugm3,O3_ugm3,CO_mgm3,CO2_mgm3,PM25_ugm3,SiteID,Lat,Long,month,day_of_week
0,8,2021-06-28,12.044255,0.085176,39.303789,0.351863,878.897362,5.072396,1,53.348754,-6.257607,6,0
1,9,2021-06-28,5.263025,9.600681,29.413390,0.350409,842.139168,7.998587,1,53.348754,-6.257607,6,0
2,9,2021-08-16,10.318074,3.006013,61.380364,0.380456,809.073556,6.122852,1,53.348754,-6.257607,8,0
3,9,2021-10-22,456.213318,NaN,NaN,0.432339,850.479574,5.752877,1,53.348754,-6.257607,10,4
4,9,2022-07-29,15.093107,NaN,NaN,0.415959,805.524520,4.757151,1,53.348754,-6.257607,7,4


In [53]:
#Export to csv

averaged_car_measurements2.to_csv('data/averaged_car_measurements.csv', index=False)